# **Webscrapping**

## Import(webscrapping)

In [ ]:
import selenium
import time
import lxml

import pandas                            as pd

from selenium.webdriver.common.by        import By
from selenium                            import webdriver
from selenium.webdriver.chrome.service   import Service
from webdriver_manager.chrome            import ChromeDriverManager
from selenium.webdriver.support.ui       import WebDriverWait
from selenium.webdriver.support          import expected_conditions as EC
from selenium.webdriver.support.ui       import Select 

from bs4                                 import BeautifulSoup


## <font color='#708090'>Código para salvar os arquivos em um dataframe - final</font>

#### quantidade aprovada e valor aprovado

In [16]:
# ======Aqui chama o site e abre o chrome navegador=======
url = "https://datasus.saude.gov.br/informacoes-de-saude-tabnet/"
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=servico)
driver.get(url)

# Aguarda até que o elemento da lista esteja presente e visível na página
toggle_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='elementor-tab-title-2152']")))
toggle_element.click()

# Aguarda até que a página carregue completamente
time.sleep(1)

# Rola a página para o elemento da lista
driver.execute_script("arguments[0].scrollIntoView(true);", toggle_element)

# Aguarda até que o link "Produção Hospitalar (SIH/SUS)" esteja presente e visível na página
link_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="elementor-tab-content-2152"]/ul/li[1]/span/a')))
driver.execute_script("arguments[0].scrollIntoView(true);", link_element)  # Rola a página para o elemento
link_element.click()

# Aguarda até que a página "Produção Hospitalar (SIH/SUS)" carregue completamente
time.sleep(1)

# Webscraping na página "Produção Hospitalar (SIH/SUS)"

## Selecionando a primeira opção "Dados Detalhados de AIH (SP), por local de internação, 2008 em diante"
# Localiza o elemento de input com o XPath desejado
input_xpath = '//*[@id="infesq"]/input[6]'
input_element = driver.find_element(By.XPATH, input_xpath)

# Localiza o elemento rádio pelo atributo "value"
radio_option_xpath = '//input[@type="radio" and @name="radiobutton" and @value=" http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/spa"]'
radio_option_element = driver.find_element(By.XPATH, radio_option_xpath)

# Clica na opção radio
radio_option_element.click()

## Selecionando a segunda opção "Brasil por municipios"
# Localiza o elemento <select> pelo id
select_element = driver.find_element(By.ID, "mySelect")

# Seleciona a segunda opção pelo texto
select = Select(select_element)
select.select_by_visible_text('Brasil por Municípios')

# ======Períodos disponíveis======= (Quantidade_aprovada)
periodos = ["Jan/2019", "Fev/2019", "Mar/2019", "Abr/2019",
            "Mai/2019", "Jun/2019", "Jul/2019", "Ago/2019",
            "Set/2019", "Out/2019", "Nov/2019", "Dez/2019",
            "Jan/2020", "Fev/2020", "Mar/2020", "Abr/2020",
            "Mai/2020", "Jun/2020", "Jul/2020", "Ago/2020",
            "Set/2020", "Out/2020", "Nov/2020", "Dez/2020",
            "Jan/2021", "Fev/2021", "Mar/2021", "Abr/2021",
            "Mai/2021", "Jun/2021", "Jul/2021", "Ago/2021",
            "Set/2021", "Out/2021", "Nov/2021", "Dez/2021",
            "Jan/2022", "Fev/2022", "Mar/2022", "Abr/2022",
            "Mai/2022", "Jun/2022", "Jul/2022", "Ago/2022",
            "Set/2022", "Out/2022", "Nov/2022", "Dez/2022",
            "Jan/2023", "Fev/2023", "Mar/2023", "Abr/2023",
            "Mai/2023", "Jun/2023", "Jul/2023", "Ago/2023",
            "Set/2023", "Out/2023", "Nov/2023", "Dez/2023"]

# Tipos de dados disponíveis
#tipos_dados = ["Quantidade_aprovada", "Valor_aprovado"]

# Inicializa um dataframe vazio para armazenar os dados
df_total = pd.DataFrame()

# Loop sobre os períodos
for periodo in periodos:
    if periodo == "Jan/2024":
        continue  # Ignora o período "Jan/2024"

    # Seleção na linha: "Município"
    select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'L')))
    select = Select(select_element)
    select.select_by_value('Município')

    # Seleção na Coluna "Grupo procedimento"
    select_element1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'C')))
    select1 = Select(select_element1)
    select1.select_by_value('Grupo_procedimento')
    
    # Seleção no Conteúdo "Quantidade aprovada"
    select_element2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'I')))
    select2 = Select(select_element2)
    #select2.select_by_value('Quantidade_aprovada') # só para quantidade aprovada
    select2.select_by_value('Valor_aprovado') # só para valor aprovado

    # Desmarca a opção "Quantidade aprovada" se o tipo de dado for "Valor_aprovado"
    if "Valor_aprovado":
        select2.deselect_by_value("Quantidade_aprovada") # só para valor aprovado
    
    # Seleção no Conteúdo "Períodos disponíveis" 
    select_element3 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'A')))
    select3 = Select(select_element3)

    # Desmarca todas as opções anteriores
    for option in select3.all_selected_options:
        select3.deselect_by_value(option.get_attribute('value'))

    # Seleciona o novo período
    select3.select_by_visible_text(periodo)

    # Aguarda até que o botão para acesso à planilha esteja disponível
    botao_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[2]/input[1]')))

    # Clica no botão para acesso à planilha
    botao_element.click()

    # Aguarda até que a nova aba esteja disponível
    nova_aba = WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    # Muda para a nova aba
    driver.switch_to.window(driver.window_handles[1])

    # Aguarda um tempo de espera implícito para garantir que a página seja carregada completamente
    driver.implicitly_wait(10)

    # Extrai o HTML da página
    html = driver.page_source
    
    # Analisa o HTML com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    
    # Encontra o elemento tbody especificado pelo XPath
    tbody_element = soup.find("table").find("tbody")
    
    # Encontra todas as linhas (tr) dentro do tbody
    linhas_tbody = tbody_element.find_all("tr")
    
    # Inicializa uma lista para armazenar os dados
    dados = []
    
    # Loop sobre as linhas dentro do tbody
    for linha in linhas_tbody:
        # Extrai as células (td) na linha
        celulas = linha.find_all("td")
        dados_linha = [celula.get_text(strip=True) for celula in celulas]
        # Adiciona os dados da linha à lista de dados
        dados.append(dados_linha)

    # Converte a lista de dados em um DataFrame do Pandas
    table = pd.DataFrame(dados)

    # Adiciona o período como uma coluna no dataframe
    table['Período'] = periodo

    # Adiciona os dados ao dataframe total
    df_total = pd.concat([df_total, table], ignore_index=True) #não pode mudar o nome do df_total

    # Fecha a aba atual
    driver.close()

    # Mudar para a aba original
    time.sleep(5)  
    driver.switch_to.window(driver.window_handles[0])
    

In [17]:
driver.quit()

In [18]:
print ("Number of Rows:{}".format(df_total.shape[0]))

Number of Rows:180354


In [19]:
df_total["Período"].unique()

array(['Jan/2019', 'Fev/2019', 'Mar/2019', 'Abr/2019', 'Mai/2019',
       'Jun/2019', 'Jul/2019', 'Ago/2019', 'Set/2019', 'Out/2019',
       'Nov/2019', 'Dez/2019', 'Jan/2020', 'Fev/2020', 'Mar/2020',
       'Abr/2020', 'Mai/2020', 'Jun/2020', 'Jul/2020', 'Ago/2020',
       'Set/2020', 'Out/2020', 'Nov/2020', 'Dez/2020', 'Jan/2021',
       'Fev/2021', 'Mar/2021', 'Abr/2021', 'Mai/2021', 'Jun/2021',
       'Jul/2021', 'Ago/2021', 'Set/2021', 'Out/2021', 'Nov/2021',
       'Dez/2021', 'Jan/2022', 'Fev/2022', 'Mar/2022', 'Abr/2022',
       'Mai/2022', 'Jun/2022', 'Jul/2022', 'Ago/2022', 'Set/2022',
       'Out/2022', 'Nov/2022', 'Dez/2022', 'Jan/2023', 'Fev/2023',
       'Mar/2023', 'Abr/2023', 'Mai/2023', 'Jun/2023', 'Jul/2023',
       'Ago/2023', 'Set/2023', 'Out/2023', 'Nov/2023', 'Dez/2023'],
      dtype=object)

## dados quantidade aprovada

In [6]:
#salvar os dados Quantidade aprovada
df_total_qa = df_total.copy()
df_total.to_csv('df_quantidade_aprovada.csv', index=False)

In [7]:
df_total_qa.head()

,0,1,2,3,4,5,6,7,8,9,Período
0,TOTAL,211,13.531.483,10.103.301,962.775,41.317,250.768,220.658,3.516.251,28.626.764,Jan/2019
1,110001 ALTA FLORESTA D'OESTE,-,-,125,7,-,-,-,51,183,Jan/2019
2,110002 ARIQUEMES,-,136,2.888,859,-,14,4,1.147,5.048,Jan/2019
3,110003 CABIXI,-,20,20,-,-,-,-,12,52,Jan/2019
4,110004 CACOAL,-,8.870,15.976,924,2,5,95,2.556,28.428,Jan/2019


## dados valor aprovado

In [20]:
df_total_va = df_total.copy()
df_total.to_csv('df_valor_aprovado.csv', index=False)

In [21]:
df_total_va.head()

,0,1,2,3,4,5,6,7,8,Período
0,TOTAL,"55.246.070,34","340.159.908,29","395.912.811,40","54.748.172,55","7.724.936,34","120.504.442,87","313.977.961,67","1.288.274.303,46",Jan/2019
1,110001 ALTA FLORESTA D'OESTE,-,"37.897,63","3.087,89",-,-,-,"408,00","41.393,52",Jan/2019
2,110002 ARIQUEMES,"1.056,29","128.329,40","99.413,85",-,"1.037,96",-,"118.467,79","348.305,29",Jan/2019
3,110003 CABIXI,-,"5.981,97",-,-,-,-,"96,00","6.077,97",Jan/2019
4,110004 CACOAL,"69.881,30","267.911,33","312.777,55","1.914,76","466,40","31.446,75","363.285,36","1.047.683,45",Jan/2019


#### qualidade aprovada

In [ ]:
df1_qa = df_raw_qa.copy()

In [ ]:
df1_qa.head()

#### valor aprovado

In [ ]:
df1_va = df_raw_va.copy()

In [ ]:
df1_va.head()

#### qualidade aprovada

In [ ]:
df1_qa.columns

In [ ]:
df1_qa = df1_qa.rename(columns={'0': 'qa_município',
                                '1': 'qa_ações_de_promoção_e_prevenção_em_saúde',
                                '2': 'qa_procedimentos_com_finalidade_diagnóstica',
                                '3': 'qa_procedimentos_clínicos',
                                '4': 'qa_procedimentos_cirúrgicos',
                                '5': 'qa_transplantes_de_orgãos_tecidos_e_células',
                                '6': 'qa_medicamentos',
                                '7': 'qa_órteses_próteses_e_materiais_especiais',
                                '8': 'qa_ações_complementares_da_atenção_à_saúde',
                                '9':'qa_total',
                                'Período': 'qa_período'})

In [ ]:
df1_qa.head()

#### valor aprovado

In [ ]:
df1_va.columns

In [ ]:
df1_va = df1_va.rename(columns={'0': 'va_município',
                                '1': 'va_procedimentos_com_finalidade_diagnóstica',
                                '2': 'va_procedimentos_clínicos',
                                '3': 'va_procedimentos_cirúrgicos',
                                '4': 'va_transplantes_de_orgãos_tecidos_e_células',
                                '5': 'va_medicamentos',
                                '6': 'va_órteses_próteses_e_materiais_especiais',
                                '7': 'va_ações_complementares_da_atenção_à_saúde',
                                '8':'va_total',
                                'Período': 'va_período'})

In [ ]:
df1_va.head()

#### qualidade aprovada

In [ ]:
print ("Number of Rows:{}".format(df1_qa.shape[0]))
print ("Number of Cols:{}".format(df1_qa.shape[1]))

#### valor aprovada

In [ ]:
print ("Number of Rows:{}".format(df1_va.shape[0]))
print ("Number of Cols:{}".format(df1_va.shape[1]))

## 2.2 Data Types

#### qualidade aprovada

In [ ]:
df1_qa.dtypes

In [ ]:
df1_va.isna().sum()

## 2.4 Fillout NA

#### qualidade aprovada

In [ ]:
df1_qa.replace('-', 0, inplace=True)

In [ ]:
df1_qa.head()

#### valor aprovado

In [ ]:
df1_va.replace('-', 0, inplace=True)


In [ ]:
df1_va.head()

## 2.5 Change Types 

### Qualidade Aprovada

In [ ]:
df1_qa.columns

In [ ]:
# Remover pontos das strings e converter para números inteiros nas colunas de 2 a 10
df1_qa.iloc[:, 1:10] = df1_qa.iloc[:, 1:10].replace('\.', '', regex=True).astype(int)


In [ ]:
df1_qa.head()

In [ ]:
# Converter para int
df1_qa['qa_ações_de_promoção_e_prevenção_em_saúde'] = df1_qa['qa_ações_de_promoção_e_prevenção_em_saúde'].astype(int)
df1_qa['qa_procedimentos_com_finalidade_diagnóstica'] = df1_qa['qa_procedimentos_com_finalidade_diagnóstica'].astype(int)
df1_qa['qa_procedimentos_clínicos'] = df1_qa['qa_procedimentos_clínicos'].astype(int)
df1_qa['qa_procedimentos_cirúrgicos'] = df1_qa['qa_procedimentos_cirúrgicos'].astype(int)
df1_qa['qa_transplantes_de_orgãos_tecidos_e_células'] = df1_qa['qa_transplantes_de_orgãos_tecidos_e_células'].astype(int)
df1_qa['qa_medicamentos'] = df1_qa['qa_medicamentos'].astype(int)
df1_qa['qa_órteses_próteses_e_materiais_especiais'] = df1_qa['qa_órteses_próteses_e_materiais_especiais'].astype(int)
df1_qa['qa_ações_complementares_da_atenção_à_saúde'] = df1_qa['qa_ações_complementares_da_atenção_à_saúde'].astype(int)
df1_qa['qa_total'] = df1_qa['qa_total'].astype(int)


In [ ]:
# Separar em mês e ano
df1_qa[['qa_mês', 'qa_ano']] = df1_qa['qa_período'].str.split('/', expand=True)

In [ ]:
df1_qa['qa_ano'] = df1_qa['qa_ano'].astype(int)

In [ ]:
# Converte o mês de texto para o número do mês
meses_dict = {'Jan': 1, 'Fev': 2, 'Mar': 3, 'Abr': 4, 'Mai': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8, 'Set': 9, 'Out': 10, 'Nov': 11, 'Dez': 12}
df1_qa['qa_dia_mês'] = df1_qa['qa_mês'].map(meses_dict)


In [ ]:
df1_qa.dtypes

In [ ]:
df1_qa.head()

In [ ]:
# Converter para números inteiros
#df1.iloc[:, 2:10] = pd.to_numeric(df1.iloc[:, 2:10].stack(), errors='coerce').fillna(0).astype(int).unstack()


### Valor aprovado

In [ ]:
df1_va.columns

In [ ]:
# Substituir os pontos de milhar por uma string vazia e trocar a vírgula por ponto
df1_va.iloc[:, 1:9] = df1_va.iloc[:, 1:9].replace('\.', '', regex=True).replace(',', '.', regex=True).astype(float)


In [ ]:
# Converter para int
df1_va['va_procedimentos_com_finalidade_diagnóstica'] = df1_va['va_procedimentos_com_finalidade_diagnóstica'].astype(float)
df1_va['va_procedimentos_clínicos'] = df1_va['va_procedimentos_clínicos'].astype(float)
df1_va['va_procedimentos_cirúrgicos'] = df1_va['va_procedimentos_cirúrgicos'].astype(float)
df1_va['va_transplantes_de_orgãos_tecidos_e_células'] = df1_va['va_transplantes_de_orgãos_tecidos_e_células'].astype(float)
df1_va['va_medicamentos'] = df1_va['va_medicamentos'].astype(float)
df1_va['va_órteses_próteses_e_materiais_especiais'] = df1_va['va_órteses_próteses_e_materiais_especiais'].astype(float)
df1_va['va_ações_complementares_da_atenção_à_saúde'] = df1_va['va_ações_complementares_da_atenção_à_saúde'].astype(float)
df1_va['va_total'] = df1_va['va_total'].astype(float)


In [ ]:
# Separar em mês e ano
df1_va[['va_mês', 'va_ano']] = df1_va['va_período'].str.split('/', expand=True)


In [ ]:
#df1_va['ano'] = df1_va['ano'].astype(int)

In [ ]:
df1_va['va_período'] = pd.to_datetime(df1_va['va_período'], format='%b/%Y')


In [ ]:
# Converte o mês de texto para o número do mês
meses_dict = {'Jan': 1, 'Fev': 2, 'Mar': 3, 'Abr': 4, 'Mai': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8, 'Set': 9, 'Out': 10, 'Nov': 11, 'Dez': 12}
df1_va['va_dia_mês'] = df1_va['va_mês'].map(meses_dict)


In [ ]:
df1_va.dtypes

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:
df1_va.head()

### Merge datasets

In [ ]:
df_agrupado = pd.concat([df1_qa, df1_va], axis=1)


In [ ]:
df_agrupado.head()

In [ ]:
df1_num=df_agrupado.select_dtypes(include=['int64', 'float64'])
df1_cat=df_agrupado.select_dtypes(exclude=['int64', 'float64', 'datetime'])

In [ ]:

# ======Aqui chama o site e abre o chrome navegador=======
url = "https://datasus.saude.gov.br/informacoes-de-saude-tabnet/"
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=servico)
driver.get(url)

# Aguarda até que o elemento da lista esteja presente e visível na página
toggle_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='elementor-tab-title-2152']")))
toggle_element.click()

# Aguarda até que a página carregue completamente
time.sleep(1)

# Rola a página para o elemento da lista
driver.execute_script("arguments[0].scrollIntoView(true);", toggle_element)

# Aguarda até que o link "Produção Hospitalar (SIH/SUS)" esteja presente e visível na página
link_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="elementor-tab-content-2152"]/ul/li[1]/span/a')))
driver.execute_script("arguments[0].scrollIntoView(true);", link_element)  # Rola a página para o elemento
link_element.click()

# Aguarda até que a página "Produção Hospitalar (SIH/SUS)" carregue completamente
time.sleep(1)

# Webscraping na página "Produção Hospitalar (SIH/SUS)"

## Selecionando a primeira opção "Dados Detalhados de AIH (SP), por local de internação, 2008 em diante"
# Localiza o elemento de input com o XPath desejado
input_xpath = '//*[@id="infesq"]/input[6]'
input_element = driver.find_element(By.XPATH, input_xpath)

# Localiza o elemento rádio pelo atributo "value"
radio_option_xpath = '//input[@type="radio" and @name="radiobutton" and @value=" http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/spa"]'
radio_option_element = driver.find_element(By.XPATH, radio_option_xpath)

# Clica na opção radio
radio_option_element.click()

## Selecionando a segunda opção "Brasil por municipios"
# Localiza o elemento <select> pelo id
select_element = driver.find_element(By.ID, "mySelect")

# Seleciona a segunda opção pelo texto
select = Select(select_element)
select.select_by_visible_text('Brasil por Municípios')

# ======Períodos disponíveis======= (Quantidade_aprovada)
periodos = ["Nov/2023", "Dez/2023"]

# Tipos de dados disponíveis
#tipos_dados = ["Quantidade_aprovada", "Valor_aprovado"]

# Inicializa um dataframe vazio para armazenar os dados
df_total = pd.DataFrame()


